# Neuromodulation example

This neuromodulation creates a small network of neurons. We also use the reaction diffusion model by Anu G Nair 2015.

To generate the ```reaction_diffusion.json``` file in ```data/str-dspn-e150602_c1_D1-mWT-0728MSN01-v20211026/``` from the xml file we run ```data/convert_sbml_to_json.sh```

To get the RxD functionality of the ```DA_syn``` we must specify the "RxD" block in the connectivity block of the network configuration. See ```data/connectivity.json```

```
    "channel_parameters":
		"RxD": {
		    "species_name": "DA",
		    "flux_variable": "open",
		    "region": "internal",
            "weight_scaling": 1e9,
		},

     ...
     }
```

## Network setup

We have two neurons. The first neuron (Neuron 0) receives external input (cortical from t=0s and DA from t=0.1s). The cortical input will activate the first neuron, and through activation of synapses on the second neuron, we will see the dopamine level increase in the second neuron (Neuron 1).

The first neuron also receives direct DA activation from external input (starting at 100ms).

In [1]:
import os
from snudda import Snudda

neuron_path = os.path.join("data", "dspn")
network_path = os.path.join("networks", "neuromodulation_example_anu_with_real_dspn")
connectivity_path = os.path.join("data", "connectivity.json")

snudda = Snudda(network_path=network_path)
si = snudda.init_tiny(neuron_paths=neuron_path, neuron_names="dspn", number_of_neurons=[2], 
                      connection_config=connectivity_path, random_seed=12345)

si.network_data["regions"]["Cube"]["neurons"]["dspn"]["reaction_diffusion"] = "reaction_diffusion.json"
si.network_data["regions"]["Cube"]["neurons"]["dspn"]["modulation"] = "test-modulation.json"
si.network_data["regions"]["Cube"]["neurons"]["dspn"]["modulation_key"] = "abc"


si.write_json()

snudda.create_network()

Reading connectivity from data/connectivity.json
Adding neurons: dspn from dir data/dspn
Writing networks/neuromodulation_example_anu_with_real_dspn/network-config.json
Writing networks/neuromodulation_example_anu_with_real_dspn/network-config.json
Placing neurons
Network path: networks/neuromodulation_example_anu_with_real_dspn
Reading SNUDDA_DATA=None from networks/neuromodulation_example_anu_with_real_dspn/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/utils/../data from networks/neuromodulation_example_anu_with_real_dspn/network-synapses.hdf5
No n_putative_points and putative_density, setting n_putative_points = 63
(this must be larger than the number of neurons you want to place)
Generating 63 points for networks/neuromodulation_example_anu_with_real_dspn/mesh/Cube-cube-mesh-2.917951293943981e-05.obj
Filtering, keeping inside points: 4 / 26
neuron_name = 'dspn_0', num = 2, neuron_path = 'data/dspn/str-dspn-e150602_c1_D1-mWT-0728MSN01-v20211026'
stop_paralle

In [2]:
input_config = os.path.join("data", "input_v2.json")
snudda.setup_input(input_config=input_config)

Setting up inputs, assuming input.json exists
Reading SNUDDA_DATA=None from networks/neuromodulation_example_anu_with_real_dspn/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/utils/../data from networks/neuromodulation_example_anu_with_real_dspn/network-synapses.hdf5
Writing input spikes to networks/neuromodulation_example_anu_with_real_dspn/input-spikes.hdf5
Reading SNUDDA_DATA=None from networks/neuromodulation_example_anu_with_real_dspn/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/utils/../data from networks/neuromodulation_example_anu_with_real_dspn/network-synapses.hdf5
!!! Warning, combining definition of cortical with cortical input for neuron dspn_0 0 (meta modified by input_config)
Writing spikes to networks/neuromodulation_example_anu_with_real_dspn/input-spikes.hdf5
stop_parallel disabled, to keep pool running.

Execution time: 0.6s


In [3]:
mech_dir = os.path.join("..", "..", "..", "..", "BasalGangliaData", "data", "neurons", "mechanisms")
sample_dt = None # 0.00005
sim = snudda.simulate(time=0, mech_dir=mech_dir, verbose=True, sample_dt=sample_dt)

Using input file networks/neuromodulation_example_anu_with_real_dspn/input-spikes.hdf5
NEURON mechanisms already compiled, make sure you have the correct version of NEURON modules.
If you delete x86_64, aarch64, arm64 directories (or nrnmech.dll) then you will force a recompilation of the modules.
Reading SNUDDA_DATA=None from networks/neuromodulation_example_anu_with_real_dspn/network-config.json
Reading SNUDDA_DATA=/home/hjorth/HBP/Snudda/snudda/utils/../data from networks/neuromodulation_example_anu_with_real_dspn/network-synapses.hdf5
Using network_file: networks/neuromodulation_example_anu_with_real_dspn/network-synapses.hdf5
Using input_file: networks/neuromodulation_example_anu_with_real_dspn/input-spikes.hdf5
Using output_file: networks/neuromodulation_example_anu_with_real_dspn/simulation/output.hdf5
Using logFile: networks/neuromodulation_example_anu_with_real_dspn/log/network-simulation-log.txt-0
Worker 0 : Loading network from networks/neuromodulation_example_anu_with_real_

AttributeError: mod_pka_g_min_kir_ms, the mechanism does not exist at dspn_0[0].axon[0](0.5)

### Add PKA and DA recordings
Here we add recordings from a compartment that receives synaptic input from its neighbour. This has been checked by looking at the synapse_connection matrix (snudda_load command).

In [ ]:
sim.add_rxd_concentration_recording(species="PKAc", neuron_id=1,
                                    region="dend_internal",
                                    sec_type="dend",
                                    sec_id=4,
                                    sec_x=0.25)

In [ ]:
sim.add_rxd_concentration_recording(species="DA", neuron_id=1,
                                    region="dend_internal",
                                    sec_type="dend",
                                    sec_id=4,
                                    sec_x=0.25)

### Add additional PKA and DA recorings, for input

In [ ]:
sim.add_rxd_concentration_recording(species="DA", neuron_id=0,
                                    region="dend_internal",
                                    sec_type="dend",
                                    sec_id=7,
                                    sec_x=0.233)

In [ ]:
# sim.add_rxd_internal_concentration_recording_all(species="PKA", neuron_id=0)

In [ ]:
sim.add_rxd_internal_concentration_recording_all_species(neuron_id=0)

In [ ]:
# Add density mechanism record
sim.add_density_mechanism_recording(neuron_id=0, sec_id=0, sec_x=0.5, density_mechanism_name="kir_ms", variable_name="modulation_factor")

In [ ]:
sim.run(t=200)

In [ ]:
sim.record.write()

## Load the data and plot

In [ ]:
from snudda.utils import SnuddaLoadSimulation

output_file = os.path.join(network_path, "simulation", "output.hdf5")
nd = SnuddaLoadSimulation(output_file)
time = nd.get_time()
data_pka = nd.get_data("PKAc", 1)[0][1]
data_da = nd.get_data("DA", 1)[0][1]
data_da_external = nd.get_data("DA", 0)[0][0]

# This is saved with add_rxd_internal_concentration_recording_all -- check that it worked 
data_pka_all0 = nd.get_data("PKAc", 0)[0][0]

In [ ]:
data_types = nd.list_data_types(0)
all_species_data = nd.get_all_data(neuron_id=0, exclude=["spikes", "voltage"])
time = nd.get_time()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
for data_type in all_species_data:
    fig.add_trace(go.Scatter(x=time, y=all_species_data[data_type][0][0].T[0], name=data_type))

fig.update_layout(xaxis_title="Time (s)", yaxis_title="Concentration", width=1000, height=800)
fig.show()

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
for data_type in all_species_data:
    plt.plot(time, all_species_data[data_type][0][0].T[0], label=data_type)
plt.xlabel("Time (s)")
plt.ylabel("Concentration")
plt.legend()
plt.show()

In [ ]:
all_species_data[data_type][0][0].T[0]

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(time, data_da, label="DA")
plt.xlabel("Time (s)")
plt.ylabel("Concentration")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(time, data_pka, label="PKAc")
plt.xlabel("Time (s)")
plt.ylabel("Concentration")
plt.legend()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(time, data_da_external, label="DA from external?")
plt.xlabel("Time (s)")
plt.ylabel("Concentration")
#plt.legend()
plt.title("DA from external?")
plt.show()

In [ ]:
from snudda.plotting import SnuddaPlotSpikeRaster2
fig_file_raster = f"spike-raster.png"

time_range_zoom = (0,0.1)
spr = SnuddaPlotSpikeRaster2(network_path=network_path)

spr.plot_spike_raster(fig_file=fig_file_raster, time_range=time_range_zoom)

In [ ]:
%matplotlib inline

sim_file = os.path.join(network_path, "simulation", "output.hdf5")

from snudda.plotting.plot_traces import PlotTraces
pt = PlotTraces(output_file=sim_file)
# Use trace_id to specify which traces
ax = pt.plot_traces(offset=0, time_range=(0,0.5),fig_size=(10,4))

## TODO!! Also add ability to have external synapses affect RxD concentrations

In [ ]:
sim.neurons[0].icell.soma[0](0.5).naf_ms.gbar

In [ ]:
sim.neurons[0].icell.soma[0](0.5).pas.g